Установим библиотеку transformers, чтобы получить доступ к реализации DistilBERT'а, а также предобученной версии модели.

In [1]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 4.9/4.9 MB 976.4 kB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 951.9 kB/s eta 0:00:00
     -------------------------------------- 163.5/163.5 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00
     ------------------------------------ 151.7/151.7 kB 900.7 kB/s eta 0:00:00
     -------------------------------------- 267.7/267.7 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Загрузим необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


В качестве датасета возьмём файл, созданный на основе датасета SST2 (https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv), импортируем его напрямую в датафрейм pandas.
Можно отобразить первые 5 строк датафрейма и посмотреть, что из себя представляют данные:

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [3]:
batch_1 = df[:]

С помощью pandas мы можем узнать, сколько предложений помечены как положительные (значение label 1) и отрицательные (значение label 0).

In [4]:
batch_1[1].value_counts()

1    3610
0    3310
Name: 1, dtype: int64

Загрузим предобученную модель. В переменной model будет предобученная модель distilBERT, tokenizer - токенизатор для предобработки данных.

In [5]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Теперь мы можем токенизировать набор данных - разбить его на компоненты понятного для модели вида. Этот код преобразует каждое предложение в список идентификаторов.

In [6]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

После токенизации tokenized представляет собой список предложений - каждое предложение представлено в виде списка токенов. Мы хотим, чтобы модель обработала все примеры разом (как один пакет). Просто так быстрее. Для этого нужно дозаполнить все списки до одинакового размера путем прибавления к более коротким векторам идентификатора 0, чтобы мы могли представлять входные данные в виде одного двумерного массива, а не списка списков (разной длины).

In [7]:
max_len = 0
min_len = float("inf")
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
    elif len(i) < min_len:
        min_len = len(i)

print(min_len, max_len)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

3 67


Датасет теперь находится в padded, мы можем просмотреть его размеры ниже.

In [8]:
np.array(padded).shape

(6920, 67)

Это ещё рано отправлять модели на вход. Нам нужно создать другую переменную, чтобы указать модели игнорировать наши дополнения, когда она обрабатывает свои входные данные.

In [9]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(6920, 67)

После запуска этого кода переменная last_hidden_states будет содержать вывод DistilBERT'а, который представляет собой 3d тензор.

In [10]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Вырежем ту часть выходных данных, которая нам нужна. Это выходные данные, соответствующие первому токену каждого предложения. Модель добавляет маркер под названием [CLS] (для классификации) в начале каждого предложения. Вырежем данные, соответствующие этому токену, для всех предложений и всех скрытых нейронов.

In [11]:
features = last_hidden_states[0][:,0,:].numpy()

Метки, указывающие, какое предложение является положительным, а какое негативным, теперь переходят в переменную labels

In [12]:
labels = batch_1[1]

Разделим датасет на обучающую и тестовую выборки.

In [13]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

Теперь мы можем создать модель логистической регрессии и обучить ее на нашем наборе данных.

In [14]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

Модель обучена, и мы можем подсчитать метрику (accuracy) на тестовой выборке.

In [15]:
lr_clf.score(test_features, test_labels)

0.8485549132947977

In [29]:
user_data = pd.read_csv("C:\\Users\\Gigabyte\\Downloads\\test_d.csv", header=None)
tokenized_user = user_data[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len_user = 0
min_len_user = float("inf")
for i in tokenized_user.values:
    if len(i) > max_len_user:
        max_len = len(i)
    elif len(i) < min_len_user:
        min_len = len(i)
padded_user = np.array([i + [0]*(max_len_user-len(i)) for i in tokenized_user.values])
attention_mask_user = np.where(padded_user != 0, 1, 0)
input_ids_user = torch.tensor(padded_user)  
attention_mask_user = torch.tensor(attention_mask_user)

with torch.no_grad():
    last_hidden_states_user = model(input_ids_user, attention_mask=attention_mask_user)
user_data= last_hidden_states_user[0][:,0,:].numpy()

lr_clf.predict(user_data)

array([1, 0, 1], dtype=int64)

Я узнал о такой разработке в задачах обработки естественного языка, как модель BERT. Также мы ознакомились с основными принципами работы модели BERT и некоторых возможностях библиотеки Scikit Learn для её применения в DS и ML.

Для решения задачи классификации текста была использована предобученная модель distilBERT (облегчённая и ускоренная версия BERT, но сравнимая с ней в результативности) и токенизатор из библитеки transformers для предобработки данных, модель логистической регрессии из библиотеки Scikit Learn.

Судя по тестированию, решение задачи получить удалось, был получен хороший результат с точностью около 85%.